# Chapter 3 - Critical Clearing Time

## Introduction

The Critical Clearing Time (CCT) is the maximum time interval by which the fault must be cleared in order to preserve the system stability.
This CCT is a measure of the system transient stability.

There are many ways to improve the transient stability of a power system.
One common way is to increase the exciter gain.

A more comprehensive review of power system stability is given in:

> N. Hatziargyriou et al., "Definition and Classification of Power System Stability – Revisited & Extended," in IEEE Transactions on Power Systems, vol. 36, no. 4, pp. 3271-3281, July 2021, doi: [10.1109/TPWRS.2020.3041774](https://ieeexplore.ieee.org/document/9286772).

## Objective

In this chapter, you will learn how to:
- find the critical clearing time of the WECC system
- improve the system transient stability by increase the exciter gain

After complete the chapter, please save and submit this jupyter notebook file in **CANVAS** with **FORMATTED** name:

`FirstName_LastName_NetID_ChID.ipynb`, for example, `Tim_Cook_tcook3_Ch3.ipynb`.

## Contingency Analysis

### Stable Case

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import logging

import andes
andes.config_logger(stream_level=50)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ss = andes.load(andes.get_case('wecc/wecc_full.xlsx'),
                default_config=True,
                setup=False,
                no_output=True)

Add ``Toggle`` for line tripping. One acts at 1.0 seconds and the other acts at 1.5 seconds.

In [ ]:
ss.add('Toggle', dict(model="Line", dev="Line_2", t=1.0))
ss.add('Toggle', dict(model="Line", dev="Line_2", t=1.5))
ss.setup()

Solve power flow

In [ ]:
ss.PFlow.run()

Turn off the ``criteria`` and run TDS to ``30``s.

In [ ]:
ss.TDS.config.criteria = 0
ss.TDS.config.tf = 30
ss.TDS.run()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))

ss.TDS.plt.plot(ss.GENROU.delta,
                ytimes=180 / np.pi,
                ylabel=r'$\delta$ [deg]',
                grid=True, title='Generator angle',
                ax=ax[0], fig=fig, show=False)

ss.TDS.plt.plot(ss.GENROU.omega,
                ytimes=60,
                ylabel=r'Freq. [Hz]',
                grid=True, title='System frequency',
                ax=ax[1], fig=fig, show=False)

From the figure above, we can see that with 0.5s fault clearing time, the system is stable.

### Unstable Case

**Exercise**: Check if the system is stable with fault clearing time being ``1``s.

In [ ]:
ss = andes.load(andes.get_case('wecc/wecc_full.xlsx'),
                default_config=True,
                setup=False,
                no_output=True)

ss.add('Toggle', dict(model="Line", dev="Line_2", t=1.0))
ss.add('Toggle', dict(model="Line", dev="Line_2", t=<**ANSWER**>))
ss.setup()

ss.PFlow.run()

ss.TDS.config.criteria = 0
ss.TDS.config.tf = 30
ss.TDS.run()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3))

ss.TDS.plt.plot(ss.GENROU.delta,
                ytimes=180 / np.pi,
                ylabel=r'$\delta$ [deg]',
                grid=False, title='Generator angle',
                ax=ax[0], fig=fig, show=False,
                hline1=-49, hline2=155,
                vline1=2.3)

ss.TDS.plt.plot(ss.GENROU.omega,
                ytimes=60,
                ylabel=r'Freq. [Hz]',
                grid=True, title='System frequency',
                ax=ax[1], fig=fig, show=False)

From the figure above, you should see that with increased fault clearing time, the maximum delta angle is larger than 180 degree, which means the system is unstable.

## Find CCT

Now, let us find the CCT of the WECC system.

**Exercise**: First, let us define a function to check if the system is stable based on the rotor angle seperation criteria.

In [ ]:
def check_stable(ss, delta_limit):
    """
    Check if the system is stable after the fault.

    Parameters
    ----------
    ss: andes.core.system.System
        ANDES system instance that have been setup
    delta_limit: float
        Maximum delta angle difference in degree
    
    Returns
    -------
    res: bool
        True if the system is stable
    """
    ss.PFlow.run()  # run power flow

    ss.TDS.config.no_tqdm = 1  # turn off the progress bar
    ss.TDS.config.tf = 20  # simulation time
    ss.TDS.config.criteria = 0  # turn off the criteria check
    ss.TDS.run()

    delta = ss.dae.ts.x[:, <**ANSWER**>]  # extract delta angle of GENROU
    diff_max = np.max(delta - np.min(delta))  # maximum delta angle difference

    res = (diff_max < np.deg2rad(delta_limit)).tolist()  # check if the system is stable

    return res

In [ ]:
def sys_modify(ss, ct, rf_exc=False):
    """
    Modify the system and set it up.

    Parameters
    ----------
    ss: andes.core.system.System
        ANDES system instance that have NOT been setup
    ct: float
        fault clearing time
    rf_exc: bool
        True to reinforce exciter

    Return
    ------
    ss: andes.core.system.System
        Modified ANDES system instance that have been setup
    """
    # --- Fault ---
    t0 = 1  # fault time
    ss.add('Toggle', dict(model="Line", dev="Line_2", t=t0))  # add fault
    # add fault clearing
    ss.add('Toggle', dict(model="Line", dev="Line_2", t=t0 + ct))

    # --- setup ---
    ss.setup()

    # --- Exciter adjustment to improvt CCT ---
    # NOTE: Skip this part for the first time
    # NOTE: Uncomment and complete the following lines
    # if rf_exc:
    #     exc_gain = <**ANSWER**>
    #     ss.EXST1.set(src= <**ANSWER**>,
    #                  attr='v',
    #                  idx=ss.EXST1.idx.v,
    #                  value= 1.2 * exc_gain)

    return ss


**Exercise**: Then, let us find the CCT using the bisection method.

In [ ]:
def cct(case, delta_limit, rf_exc=False):
    """
    Find the critical clearing time for the system by bisection method.

    Parameters
    ----------
    case: str
        Case name
    delta_limit: float
        Maximum delta angle difference in degree
    rf_exc: bool
        True to reinforce exciter
    
    Returns
    -------
    ct: float
        Critical clearing time
    ss: andes.core.system.System
        ANDES system instance that run the simulation
    """
    # --- initialization ---
    ct = 1  # clearing time
    ct0 = -1  # previous clearing time
    ct0_s = 0  # previous stable clearing time
    ct0_u = 2 * ct  # previous unstable clearing time
    res = False  # result
    n_iter = 0  # iteration counter

    # --- bisection method ---
    print("Exciter reinforcement: %r" % rf_exc)
    while n_iter < 100:  # max iterations
        ss = andes.load(case=case, default_config=True,
                        setup=False, no_output=True)
        ss = sys_modify(ss=ss, ct=ct, rf_exc=rf_exc)
        res = check_stable(ss=ss, delta_limit=delta_limit)

        # --- bisection method ---
        ct0 = ct
        if res: # stable, increase ct
            ct0_s = ct
            ct = 0.5 * (<**ANSWER**>)
        else:  # unstable, reduce ct
            ct0_u = ct
            ct = 0.5 * (<**ANSWER**>)

        # --- exit condition ---
        ct_span = np.abs(ct0_u - ct0_s)
        print("Iter %d: CT=%f sec; Stable=%r" % (n_iter, ct, res))
        n_iter += 1
        if res & (ct_span > 0) & (ct_span < 1e-3):  # convergence
            return ct, ss
    return ct, ss


Find the CCT of the WECC system.

In [ ]:
ct, ssr = cct(case=andes.get_case('wecc/wecc_full.xlsx'),
              delta_limit=180, rf_exc=False)


In [ ]:
ct

## Improve the transient stability

There are many ways to improve the transient stability of a power system.
One common way is to increase the exciter gain.

You can inspect the existing ``Exciter`` devices in the system. As shown below, there are three types of exciter in the WECC system.

The block diagrams of ``EXST1_PTI`` can be found at [Exciter Model: EXST1_PTI](https://www.powerworld.com/WebHelp/Content/TransientModels_HTML/Exciter%20EXST1_PTI.htm?tocpath=Transient%20Stability%20Add-On%20(TS)%7CTransient%20Models%7CGenerator%7CExciter%7C_____66).


In [ ]:
ss.Exciter.models

Now, you can improve the system transient stability by increasing the exciter gain.
In the above code block function ``sys_modify``, you need to complete the code block.

After updating the function, rerun that function code block to make it effective (or you also can rerun the entire notebook).

Find the critical clearing time of the improved system

In [ ]:
ct2, ssr2 = cct(case=andes.get_case('wecc/wecc_full.xlsx'),
                delta_limit=180, rf_exc=True)


In [ ]:
ct2

In [ ]:
ct2 - ct

Cleanup

In [ ]:
!andes misc -C